In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [9]:
# 이번에 여러 파일을 불러오겠습니다.

raw = pd.DataFrame()
raw.head()

file1 = r'.\rawfiles\CARD_SUBWAY_MONTH_201901.csv'
file2 = r'.\rawfiles\CARD_SUBWAY_MONTH_201902.csv'
file3 = r'.\rawfiles\CARD_SUBWAY_MONTH_201903.csv'
file4 = r'.\rawfiles\CARD_SUBWAY_MONTH_201904.csv'
file5 = r'.\rawfiles\CARD_SUBWAY_MONTH_201905.csv'
file6 = r'.\rawfiles\CARD_SUBWAY_MONTH_201906.csv'

temp1 = pd.read_csv(file1)
temp2 = pd.read_csv(file2)
temp3 = pd.read_csv(file3)
temp4 = pd.read_csv(file4)
temp5 = pd.read_csv(file5)
temp6 = pd.read_csv(file6)

# pandas concat 합치때, 인덱스 번호 그래로 간다. 옵션 ignore_index=True 사용하면 인덱스 새로 생성
raw = pd.concat([temp1, temp2, temp3, temp4, temp5, temp6], axis=0)

# 승하자총승객수
raw['승하자총승객수'] = raw['승차총승객수'] + raw['하차총승객수']

# 사용일자의 데이터 타입을 int에서 datetime으로 바꿔준다.
raw['사용일자'] = pd.to_datetime(raw['사용일자'], format='%Y%m%d')

# Monday=0, Sunday=6
raw['사용일자'].dt.dayofweek

# 사용일자에 대해서 요일로 변환
raw['요일'] = raw['사용일자'].dt.dayofweek

conv = {
    0 : '일',
    1 : '월',
    2 : '화',
    3 : '수',
    4 : '목',
    5 : '금',
    6 : '토'
}

# 숫자 요일을 한글명 요일로 변경
raw['요일'] = raw['요일'].map(conv)

# 사용일자 컬럼에서 년월 값을 가져와 년월 컬럼 만들자

raw['연월'] = raw['사용일자'].dt.strftime('%Y%m')

# 사용일자 컬럼에서 년월 값을 가져와 년월 컬럼 만들자

raw['월일'] = raw['사용일자'].dt.strftime('%m%d')

In [10]:
rawmonth=raw[['노선명','역명','승하자총승객수','연월']]
rawmonth.head()

,노선명,역명,승하자총승객수,연월
0,2호선,을지로4가,7590.0,201901
1,3호선,을지로3가,15658.0,201901
2,2호선,을지로입구,43808.0,201901
3,2호선,시청,14430.0,201901
4,6호선,동묘앞,16549.0,201901


In [11]:
x1=rawmonth[rawmonth['연월']=='201901']['승하자총승객수']
x2=rawmonth[rawmonth['연월']=='201902']['승하자총승객수']
x3=rawmonth[rawmonth['연월']=='201903']['승하자총승객수']
x4=rawmonth[rawmonth['연월']=='201904']['승하자총승객수']
x5=rawmonth[rawmonth['연월']=='201905']['승하자총승객수']
x6=rawmonth[rawmonth['연월']=='201906']['승하자총승객수']

In [12]:
print(x1.shape,x2.shape,x3.shape,x4.shape,x5.shape,x6.shape)
# 각 그룹의 표본수 N > 30 이므로 정규성을 만족한다고 가정

(18334,) (16543,) (18342,) (17749,) (18336,) (17724,)


In [13]:
# Scipy.stats를 사용한 일원분산분석
import scipy.stats as stats
F_statistic, pVal = stats.f_oneway(x1,x2,x3,x4,x5,x6)
print('데이터의 일원분산분석 결과 : F={0:.1f}, p={1:.5f}'.format(F_statistic, pVal))
if pVal < 0.05:
    print('P-value 값이 충분히 작음으로 인해 그룹의 평균값이 통계적으로 유의미하게 차이납니다.')

데이터의 일원분산분석 결과 : F=48.3, p=0.00000
P-value 값이 충분히 작음으로 인해 그룹의 평균값이 통계적으로 유의미하게 차이납니다.


In [49]:
# Statsmodel을 사용한 일원분산분석
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
model = ols('승하자총승객수 ~ C(연월)', rawmonth).fit()
anova_lm(model)
# P < 0.05 이므로 귀무가설 기각, 5월은 다른 월과 승차총승객수의 유의미한 차이가 있다.

,df,sum_sq,mean_sq,F,PR(>F)
C(연월),5.0,1.667172e+11,3.334344e+10,48.273302,4.463863e-50
Residual,107022.0,7.392248e+13,6.907222e+08,NaN,NaN


In [53]:
# 사후분석

# 본페로니 교정
    # 특징 : 응용 범위가 넓음(모수,비모수 적용 가능)
    # 장점 : ANOVA, 다중 t-test, 비모수 검정 등에 적용 가능
    # 단점 : 비교대상이 많아질수록 검정력이 약해짐
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

comp = MultiComparison(rawmonth.승하자총승객수,rawmonth.연월)

In [51]:
result = comp.allpairtest(scipy.stats.ttest_ind, method='bonf')
print(result[0])
# 3월과 6월, 4월과 5월을 제외하면 모든 월의 승차총승객수의 유의미한 차이가 있다.
# 왜?

Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.00, alphacBonf=0.003
 group1  group2   stat    pval  pval_corr reject
------------------------------------------------
2019-01 2019-02   4.2759    0.0    0.0003   True
2019-01 2019-03  -4.7257    0.0       0.0   True
2019-01 2019-04  -8.5561    0.0       0.0   True
2019-01 2019-05  -8.4655    0.0       0.0   True
2019-01 2019-06  -2.9358 0.0033    0.0499   True
2019-02 2019-03  -8.8812    0.0       0.0   True
2019-02 2019-04 -12.5644    0.0       0.0   True
2019-02 2019-05 -12.5007    0.0       0.0   True
2019-02 2019-06  -7.1563    0.0       0.0   True
2019-03 2019-04  -3.8865 0.0001    0.0015   True
2019-03 2019-05  -3.7615 0.0002    0.0025   True
2019-03 2019-06   1.8006 0.0718       1.0  False
2019-04 2019-05   0.1525 0.8788       1.0  False
2019-04 2019-06   5.6737    0.0       0.0   True
2019-05 2019-06   5.5598    0.0       0.0   True
------------------------------------------------


In [52]:
#투키의 HSD - Tuckey's Honestly Significant Difference = "진정으로 유의미한 차이"
    # 초기 투키 테스트(Tukey's test)가 동일한 샘플수에서의 비교를 전제한데에 
    # 대하여 나중에 투키-크레이머 방법(Tukey-Kramer method)으로 서로 다른 샘플수에서도 비교 가능하도록 보완되었다
from statsmodels.stats.multicomp import pairwise_tukeyhsd
hsd = pairwise_tukeyhsd(rawmonth['승하자총승객수'], rawmonth['연월'], alpha=0.05)
hsd.summary()
# 동일 결과 확인

group1,group2,meandiff,p-adj,lower,upper,reject
2019-01,2019-02,-1188.3972,0.0004,-1991.5265,-385.2678,True
2019-01,2019-03,1295.0194,0.0,512.8682,2077.1706,True
2019-01,2019-04,2384.0825,0.0,1595.4268,3172.7382,True
2019-01,2019-05,2340.9615,0.0,1558.7463,3123.1767,True
2019-01,2019-06,800.8965,0.0442,11.9583,1589.8348,True
2019-02,2019-03,2483.4166,0.0,1680.3703,3286.4628,True
2019-02,2019-04,3572.4797,0.0,2763.0968,4381.8626,True
2019-02,2019-05,3529.3587,0.0,2726.2501,4332.4673,True
2019-02,2019-06,1989.2937,0.0,1179.6355,2798.9519,True
2019-03,2019-04,1089.0631,0.0012,300.492,1877.6342,True
